In [6]:
import cv2
import numpy as np
import os

def get_orientation(contour):
	(x, y), (MA, ma), angle = cv2.fitEllipse(contour)
	return angle

red_pixels_path = '../data/Red_Pixels.txt'
ellipse_path = '../data/Ellipse.txt'
data_path = '../data/data.txt'

red_pixels_exist = os.path.exists(red_pixels_path)
ellipse_exist = os.path.exists(ellipse_path)
data_exist = os.path.exists(data_path)

if red_pixels_exist:
    open(red_pixels_path, 'w').close()

if ellipse_exist:
    open(ellipse_path, 'w').close()

if data_exist:
    open(data_path, 'w').close()

image=1
while image<601:
	img = cv2.imread("../data/Zoom/{}.jpg.jpg".format(image))
	img_suave = cv2.GaussianBlur(img, (5, 5), 0)
	img_hsv = cv2.cvtColor(img_suave, cv2.COLOR_BGR2HSV)

	lower_red = np.array([0, 200, 50])
	upper_red = np.array([10, 255, 255])
	mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

	lower_red = np.array([170, 200, 50])
	upper_red = np.array([180, 255, 255])
	mask2 = cv2.inRange(img_hsv, lower_red, upper_red)

	mask = cv2.bitwise_or(mask1, mask2)
	kernel = np.ones((5,5), np.uint8)
	opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
	num_pixels = cv2.countNonZero(opening)
#	print("Number of Red Pixels: ",num_pixels)

	with open('../data/Red_Pixels.txt', 'a') as f:
		f.write(f'{num_pixels}\n')
        
	angle_array=[]
	angle=None
	contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	img_contours = img.copy()
	for contour in contours:
		area = cv2.contourArea(contour)
#		print(area)
		if area < 1000 or area > 10000:
			continue
		ellipse = cv2.fitEllipse(contour)
		cv2.ellipse(img_contours, ellipse, (0,255,0), 2)
		angle = get_orientation(contour)
#	print('Ellipse Inclination Angle: ', angle)

	with open('../data/Ellipse.txt', 'a') as f:
		f.write(f'{angle}\n')
	cv2.imwrite("../data/Ellipse/{}_ellipse.jpg".format(image), img_contours)
    
	with open('../data/data.txt', 'a') as f:
		f.write(f'imagem: {image} + {num_pixels} + {area} + {angle} \n')
	image=image+1

print("Done")

Done


In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cv2

with open('../data/Ellipse.txt', 'r') as fid:
    data_ellipse = np.genfromtxt((line.replace('None', '-1') for line in fid), dtype=float)

data_ellipse = np.ma.masked_equal(data_ellipse, -1)
num_linhas = 24
num_colunas = 25

ellipse = np.zeros((num_linhas, num_colunas))

indice = 0
for linha in range(num_linhas):
    if linha % 2 == 0:
        ellipse[linha, :min(num_colunas, len(data_ellipse)-indice)] = data_ellipse[indice:indice + num_colunas]
    else:
        ellipse[linha, :min(num_colunas, len(data_ellipse)-indice)] = data_ellipse[indice:indice + num_colunas][::-1]
    indice += num_colunas

ellipse = np.ceil(ellipse).astype(int)
    
num_colors = 256
color_center = [0, 0, 1]
color_extremes = [1, 1, 0]

color_map = np.column_stack([
    np.linspace(color_extremes[0], color_center[0], num_colors),
    np.linspace(color_extremes[1], color_center[1], num_colors),
    np.linspace(color_extremes[2], color_center[2], num_colors)
])

color_map_with_white = np.vstack([[1, 1, 1], color_map, np.flipud(color_map)])

x_values = np.arange(1, 25, dtype=float)
y_values = np.arange(1, 24, dtype=float)
x_values *= 1.0196 * 10**-3 * 200
y_values *= 1.0196 * 10**-3 * 200

plt.imshow(ellipse, extent=[min(x_values), max(x_values), min(y_values), max(y_values)], cmap=LinearSegmentedColormap.from_list('custom', color_map_with_white))
plt.clim(0, 180)

cbar = plt.colorbar(ticks=[0, 45, 90, 135, 180])
cbar.set_label('Ellipse Angles (degrees)')
cbar.ax.set_yticklabels(['0', '45', '90', '135', '180'])

plt.xlabel('Horizontal Movements [mm]')
plt.ylabel('Vertical Movements [mm]')
plt.yticks([1, 2, 3, 4])
plt.box(on=True)
plt.gca().spines['top'].set_linewidth(2)
plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['right'].set_linewidth(2)
plt.savefig('../results/Ellipse_Angle_Variation.png')
plt.close()

print("Done")

Done


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cv2

with open('../data/Red_Pixels.txt', 'r') as fid:
    data_red_pix = np.genfromtxt((line.replace('None', '-1') for line in fid), dtype=float)

data_red_pix = np.ma.masked_equal(data_red_pix, -1)
num_linhas = 24
num_colunas = 25

red_pix = np.zeros((num_linhas, num_colunas))

indice = 0
for linha in range(num_linhas):
    if linha % 2 == 0:
        red_pix[linha, :min(num_colunas, len(data_red_pix)-indice)] = data_red_pix[indice:indice + num_colunas]
    else:
        red_pix[linha, :min(num_colunas, len(data_red_pix)-indice)] = data_red_pix[indice:indice + num_colunas][::-1]
    indice += num_colunas

x_values = np.arange(1, 25, dtype=float)
y_values = np.arange(1, 24, dtype=float)
x_values *= 1.0196 * 10**-3 * 200
y_values *= 1.0196 * 10**-3 * 200
    
plt.set_cmap('turbo')
plt.imshow(red_pix, extent=[min(x_values), max(x_values), min(y_values), max(y_values)])
plt.colorbar()
plt.xlabel('Horizontal Movements [mm]')
plt.ylabel('Vertical Movements [mm]')
plt.xlabel('Horizontal Movements [mm]')
plt.ylabel('Vertical Movements [mm]')
plt.yticks([1, 2, 3, 4])
plt.box(on=True)
plt.gca().spines['top'].set_linewidth(2)
plt.gca().spines['bottom'].set_linewidth(2)
plt.gca().spines['left'].set_linewidth(2)
plt.gca().spines['right'].set_linewidth(2)
plt.savefig('../results/Red_Pixels_Variation.png')
plt.close()

print("Done")

Done
